<a href="https://colab.research.google.com/github/ThisIsFarhan/pytorch-codes/blob/main/10_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
def tokenize(text):
  text = text.lower()
  text = text.replace("?","")
  text = text.replace("'","")
  return text.split()

In [3]:
tokenize("My name is Farhan")

['my', 'name', 'is', 'farhan']

In [8]:
vocab = {"<UNK>":0}

In [9]:
def build_vocab(row):
  tokenize_ques = tokenize(row["question"])
  tokenize_ans = tokenize(row["answer"])
  merged_tokens = tokenize_ques + tokenize_ans

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [10]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [11]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [12]:
len(vocab)

324

In [13]:
def text_to_indices(text, vocab):
  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab["<UNK>"])
  return indexed_text

In [16]:
text_to_indices("What's for dinner", vocab)

[0, 39, 0]

In [18]:
import torch
from torch.utils.data import DataLoader, Dataset

In [19]:
class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, idx):
    num_question = torch.tensor(text_to_indices(self.df.iloc[idx]["question"], self.vocab))
    num_answer = torch.tensor(text_to_indices(self.df.iloc[idx]["answer"], self.vocab))
    return num_question, num_answer

In [20]:
dataset = QADataset(df, vocab)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [21]:
dataset[1]

(tensor([1, 2, 3, 4, 5, 8]), tensor([9]))

In [24]:
len(dataloader)

90

In [25]:
len(dataset)

90

In [26]:
df.shape

(90, 2)

In [33]:
import torch.nn as nn
class network(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64,vocab_size)

  def forward(self, question):
    emb_ques = self.embedding(question)
    hidden, final = self.rnn(emb_ques)
    output = self.fc(final.squeeze(0))
    return output

In [34]:
learning_rate = 0.001
epochs = 20
criterion = nn.CrossEntropyLoss()
model = network(len(vocab))
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [36]:
for epoch in range(epochs):
  total_loss = 0
  for question,answer in dataloader:
    optimizer.zero_grad()
    op = model(question)
    loss = criterion(op, answer[0])
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 524.265888
Epoch: 2, Loss: 454.352786
Epoch: 3, Loss: 374.944297
Epoch: 4, Loss: 313.960440
Epoch: 5, Loss: 262.349965
Epoch: 6, Loss: 214.339870
Epoch: 7, Loss: 169.716900
Epoch: 8, Loss: 132.075943
Epoch: 9, Loss: 101.574808
Epoch: 10, Loss: 77.500035
Epoch: 11, Loss: 59.629891
Epoch: 12, Loss: 46.670032
Epoch: 13, Loss: 36.933269
Epoch: 14, Loss: 30.228250
Epoch: 15, Loss: 25.020453
Epoch: 16, Loss: 21.090640
Epoch: 17, Loss: 17.887947
Epoch: 18, Loss: 15.359534
Epoch: 19, Loss: 13.504330
Epoch: 20, Loss: 11.794264


In [57]:
def predict(model, question, threshold=0.5):
  indexed_ques = text_to_indices(question, vocab)
  ques_tensor = torch.tensor(indexed_ques).unsqueeze(0)
  op = model(ques_tensor)
  print(op)
  print(op.shape)
  probabilities = torch.nn.functional.softmax(op, dim=1)
  v, i = torch.max(probabilities, dim=1)
  if v >= threshold:
    print(list(vocab.keys())[i])

In [59]:
predict(model, "What is the boiling point of water in Celsius?")

tensor([[-3.2883, -3.7791, -3.2535, -3.2959, -3.6165, -2.7816,  0.1742,  0.3360,
         -3.9651, -0.2657, -4.0499, -3.6165, -3.6177, -3.4822, -3.3425, -3.1101,
         -0.0263, -3.4459, -2.7936, -3.7678, -3.9721, -3.0805, -2.9490,  0.0239,
         -3.7135, -3.7813, -3.4656, -2.9809,  7.2046, -3.2107, -3.4026, -2.8993,
         -0.0497, -3.6195, -3.4754, -2.6599, -0.3775, -3.2994, -3.4201, -3.5088,
         -3.5503, -1.1641, -3.2103, -3.6991, -3.1849, -2.9817, -3.3184, -3.5107,
         -3.4180,  0.1441, -3.7863, -3.9089, -0.7094, -0.0924, -0.5594, -3.5361,
         -3.2899, -3.7615, -1.3161, -3.3540, -3.5576,  1.6599, -3.6484, -3.5924,
         -3.7168, -0.6727, -3.3833, -3.0543,  0.4852, -3.1692, -2.8520, -2.6634,
         -0.2547, -3.7642,  1.4010, -2.7970, -4.0299,  1.0036, -3.6151, -3.4828,
         -3.5202, -3.5187, -3.1107, -3.4638, -3.4173, -0.2942, -3.1626, -3.2303,
         -3.5434, -3.0501, -3.3012, -0.7349, -3.0332, -3.4377, -3.4696,  0.2825,
         -3.3060, -3.3166, -